In [ ]:
working_df = pd.read_csv('post_wiki_scrape-Copy1.csv')
working_df = working_df.set_index(['start_year', 'title'])
working_df.head()

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
import json
from inspect import currentframe, getframeinfo

### Read in provided data

In [ ]:
data_imports = {}

# Importing all the provided CSVs and TSVs to a dictionary
for file in os.listdir("zippedData"):
    print(file)
    if file[-3:] == 'csv':
        data_imports[file[:-4]] = pd.read_csv('zippedData/' + str(file))
    elif file[-3:] == 'tsv':
        data_imports[file[:-4]] = pd.read_csv('zippedData/' + str(file), sep = '\t', encoding= 'unicode_escape')

print("Finished import")

### Starting with Bom.Movie_Gross, I want to set the index as release year and then title

In [ ]:
bom_movie_gross = data_imports['bom.movie_gross']
# bom_movie_gross[bom_movie_gross['title'].duplicated(keep=False)]

In [ ]:
bom_movie_gross = bom_movie_gross.rename(columns = {'year': 'start_year'})
bom_movie_gross = bom_movie_gross.set_index(['start_year', 'title'])
bom_movie_gross.head()

## In an effort to explore the data, we'll merge title.basics with title.ratings into imdb_df



In [ ]:
title_basics = data_imports['title.basics']
title_ratings = data_imports['title.ratings']

title_basics = title_basics.set_index('tconst')
title_ratings = title_ratings.set_index('tconst')
imdb_df = title_basics.join(title_ratings, on='tconst')


## Merge imdb_df with tmdb.movies

In [ ]:
tmdb_movies = data_imports['tmdb.movies']
tmdb_movies = tmdb_movies.drop_duplicates(subset = ['id'])
tmdb_movies['start_year'] = tmdb_movies['release_date'].apply(lambda x: int(x[:4]))
tmdb_movies = tmdb_movies.set_index(['start_year', 'title'])
tmdb_movies.head()

In [ ]:
imdb_df = imdb_df.rename(columns = {'primary_title': "title"})
imdb_df = imdb_df.reset_index()
working_df = imdb_df.set_index(['start_year', 'title']).join(tmdb_movies, how='outer', lsuffix='_imdb', rsuffix='_mvdb')


working_df = working_df.drop(columns=['Unnamed: 0', 'vote_average', 'vote_count', 'popularity', 'genre_ids', 'id'])
# working_df = working_df.fillna(value = {'genres': 'Unknown'})

working_df.head()


## Add in bom_movie_gross

In [ ]:
working_df = working_df.join(bom_movie_gross, how='outer')

# Add in tn_movie_budgets

In [ ]:
tn_movie_budgets = data_imports['tn.movie_budgets']

In [ ]:
# Find the number of movies we'll have to work with if we join working_df and tn_movie_budgets

# movie_titles_working = []
# movie_titles_tn = []
# for year, title in working_df.index:
#     movie_titles_working.append(title)

# for title in tn_movie_budgets['movie']:
#     movie_titles_tn.append(title)
    
# len(set(movie_titles_working) & set(movie_titles_tn))

In [ ]:
working_df['release_date'] = pd.to_datetime(working_df['release_date'])
tn_movie_budgets['start_year'] = pd.DatetimeIndex(tn_movie_budgets['release_date']).year
tn_movie_budgets = tn_movie_budgets.rename(columns={'movie':'title'})
tn_movie_budgets = tn_movie_budgets.set_index(['start_year', 'title'])

In [ ]:
# tn_movie_budgets.head()
working_df = working_df.join(tn_movie_budgets, how='outer', lsuffix='_imdb', rsuffix='_tn')


Drop movies from before 2000

In [ ]:
working_df = working_df.reset_index()
working_df = working_df[working_df['start_year'] > 2000]
working_df = working_df.set_index(['start_year', 'title'])

Drop NaN genres

In [ ]:
working_df = working_df.dropna(subset=['genres'])
working_df = working_df[working_df['original_language'] == 'en']
working_df.info()

# Scraping wikipedia for production budgets

At this point I had to decide whether it was better to remove foreign films from the dataset or spend the time it would require to write code to account for each individual foreign currency found through scrapping. I ultimately decided, both due to the scope of the project (Microsoft being an American company) and the marginal number of foreign films remaining in my dataset that it was better to simply return None for budgets not in US dollars.

In [ ]:
# Takes in the string, isolated as a budget, and returns a number of type int


def convert_budget_to_int(budget, debug):

    #     Checks for various non-alphanumeric characters

    if type(budget) == int:
        return budget
    if type(budget) == float:
        return int(budget)
    if budget.startswith('$CAD'):
        return None
    if budget.startswith('$') or budget.startswith('US$'):
        if budget[0] == '<':
            budget = budget[1:]
            
#  Enable for Avatar 2009, breaks other movies


#         if '$' in budget[3:]:
#             temp = budget[3:]
#             budget = budget[:temp.index('$')+3]
        while '[' in budget:
            budget = budget.replace(
                budget[budget.index('['):budget.index(']')+1], '')
        if '–' in budget:
            currency = ''
            for i in budget:
                if not i.isnumeric():
                    currency = currency + i
                else:
                    break
            if debug:
                print(budget, "Middle of dash check")
            budget = budget[budget.index('–')+1:]
            budget = currency + budget
        if '-' in budget:
            currency = ''
            for i in budget:
                if not i.isnumeric():
                    currency = currency + i
                else:
                    break
            if debug:
                print(budget, "Middle of dash check")
            budget = budget[budget.index('-')+1:]
            budget = currency + budget
        if '—' in budget:
            currency = ''
            for i in budget:
                if not i.isnumeric():
                    currency = currency + i
                else:
                    break
            if debug:
                print(budget, "Middle of dash check")
            budget = budget[budget.index('—')+1:]
            budget = currency + budget
        if debug:
            print(budget)

#     Using regex because for some reason ' ' would not be recognized for certain movies
        if bool(re.search(r"\s", budget)):
            whitespace_index = re.search(r"\s", budget).start()
            if budget[whitespace_index-1].isnumeric():
                number = budget[:whitespace_index]
                word = budget[whitespace_index+1:]
            else:
                whitespace_index = re.search(
                    r"\s", budget[:whitespace_index]+budget[whitespace_index+1:]).start()
                number = budget[:whitespace_index+1]
                word = budget[whitespace_index+2:]
            if debug:
                print(word)
                print(number)

        else:
            number, word = budget, ''
        if debug:
            print(number, 'Before \'.\' check')
        if '.' in number:
            left, right = number.split('.')
            decimal_places = len(right)
            number = number.replace('.', '')

#     Replacing instnaces of million and crore (Indian for ten million) with the proper number of zeroes

        if 'crore' in word.lower():
            try:
                number = number + '0000000'[decimal_places:]
            except:
                number = number + '0000000'
        elif 'million' in word.lower():
            try:
                number = number + '000000'[decimal_places:]
            except:
                number = number + '000000'

        if ',' in number:
            number = number.replace(',', '')

        budget = budget.strip()
        if debug:
            print(budget)

        if budget[0] == '$':
            number = number.replace('$', '')
        elif budget[:3] == 'US$':
            number = number.replace('US$', '')

        return int(number)

# Replaces spaces in a URL with %20


def urlify(in_string):
    return "%20".join(in_string.split())


'''Uses Wikipedia's API to search for movies. 
In practice I would search by the title and year to reduce the chance of an incorrect match'''


def wiki_search(search):
    url = "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=&list=search&srsearch={}".format(
        urlify(search))
    response = requests.get(url=url)

    try:
        return(response.json()['query']['search'][0]['pageid'])
    except IndexError:
        return None


'''The called function which managed the search for movies on Wikipedia, 
the isolating of the budget string, and ultimately the return of the budget as an integer'''


def wiki_grab(search, debug=False):
    searches_to_ignore = ['#Stuck 2014',
                          'House of Black Wings 2010',
                          'Restoring a Masterpiece: The Renovation of Eastman Theatre 2010',
                          'Avatar: Special Edition 2010',
                          'The Forgotten Jewel 2010',
                          'Birth of a Party 2011'
                          ]
    if search[0] == '#':
        return None
    if search in searches_to_ignore:
        return None
    pageid = wiki_search(search)
    if debug:
        print(pageid)
    if pageid is None:
        return None
    url = 'https://en.wikipedia.org/w/api.php?action=parse&format=json&pageid={}&prop=text&formatversion=2'.format(
        pageid)
    if debug:
        print(url)
    response = requests.get(url=url)
    soup = BeautifulSoup(response.json()['parse']['text'])

    if 'Budget</th>' in str(soup):
        if soup.find(text='Budget').next.text:
            return convert_budget_to_int(soup.find(text='Budget').next.text, debug)
        elif '(gross)' in soup.find(text='Budget').next.text:
            gross = soup.find('li', text=re.compile(r' .+(\(gross\))')).text
            gross = gross.replace(' (gross)', '')
            gross = convert_budget_to_int(gross, debug)
            return(gross)
        elif re.compile(r' \d') in soup.find(text='Budget').next.li.text:
            return(soup.find('li', text=re.compile(r' \d')))

# A test run
# print(wiki_grab("Habermann 2010", True))


In [ ]:
working_df['budget_wiki'] = np.nan
for year, title in working_df[working_df['budget_wiki'].isna()].index.values.tolist():
    try:
#         Do not search again in the senario that we're running this code multiple times due to error
        if working_df.loc[(year, title), 'budget_wiki'].values[0] == -1:
            continue
        print(title, year)
        budget = wiki_grab(title + ' ' + str(year))
        if budget == None:
            working_df.loc[(year, title), 'budget_wiki'] = -1
        else:
            working_df.loc[(year, title), 'budget_wiki'] = budget
        print(working_df.loc[(year, title), 'budget_wiki'])
    except:
        working_df.loc[(year, title), 'budget_wiki'] = -1

Save my work

In [ ]:
# working_df.to_csv('post_wiki_scrape.csv')

working_df = pd.read_csv('post_wiki_scrape.csv')
working_df = working_df.set_index(['start_year', 'title'])
working_df.head()

In [ ]:
# Convert financial columns to int64 for comparison

def convert_columns_to_int(budget):
    try:
        return int(float(budget))
    except:
        return convert_budget_to_int(budget, False)

for column in ['domestic_gross_imdb', 'foreign_gross', 'production_budget', 'domestic_gross_tn', 'worldwide_gross', 'budget_wiki']:
    working_df[column] = working_df[column].fillna(-1)
    working_df[column] = working_df[column].apply(lambda x: convert_columns_to_int(x))
    working_df[column] = working_df[column].astype('int64')

working_df.info()

In [ ]:
'''
Derive a working worldwide gross and working budget based on the data gathered with a
preference for information from IMDB and Wikipedia based on token examinations of the datasets
'''

working_df['working_wwg'] = working_df.apply(
    lambda x: x['domestic_gross_imdb'] + x['foreign_gross'] if x['domestic_gross_imdb'] > -1 and x['foreign_gross'] > -1 
        else x['worldwide_gross'], axis = 1)
working_df['working_budget'] = working_df.apply(
    lambda x: x['budget_wiki'] if x['budget_wiki'] > -1 else x['production_budget'], axis = 1)

In [ ]:
working_df['roi'] = working_df.apply(lambda x: x['working_wwg'] - x['working_budget'], axis = 1)
working_df = working_df.drop_duplicates()

# Split 'genre' column into genre1, genre2, genre3


In [ ]:
# Find the maximum amount of commas in the genres column
comma_counter = 0

for each in working_df['genres']:
    if type(each) == str:
        current_count = each.count(',')
        if current_count > comma_counter:
            comma_counter = current_count
    
comma_counter

In [ ]:
working_df[['genre1', 'genre2', 'genre3']] = working_df['genres'].str.split(',', expand=True)


## replacing NaN with "Unknown"

In [ ]:
working_df = working_df.fillna(value = {
    'genres': 'Unknown', 'genre1': 'Unknown', 'genre3': 'Unknown', 'genre2': 'Unknown'
})

analysis_df = working_df[[
    'runtime_minutes',
    'genres',
    'genre1',
    'genre2',
    'genre3',
    'working_wwg',
    'working_budget',
    'roi'
]].copy()
analysis_df.head()

In [ ]:
analysis_df = analysis_df[analysis_df['roi'] != 0]
analysis_df = analysis_df[analysis_df['working_wwg'] != -1]
analysis_df = analysis_df[analysis_df['working_budget'] != -1]
analysis_df.info()

In [ ]:
analysis_df = analysis_df.reset_index()
analysis_df= analysis_df.rename(columns = {'start_year': 'year'})

# Analysis

## We'll break up the dataframe along certain budget markers and create series through which we can analyse the performance of different genres

Note that this method double and tripple counts certain movies with more than one listed genre, which is why it's important for us to eliminate genres with too few examples to prevent skewing.

In [ ]:
genre_roi = pd.concat([pd.Series(analysis_df['roi'].values, analysis_df['genre1']),
                       pd.Series(analysis_df['roi'].values, analysis_df['genre2']),
                       pd.Series(analysis_df['roi'].values, analysis_df['genre3'])])

analysis_df_1mil = analysis_df[analysis_df['working_budget'] >= 1000000].copy()

genre_roi1 = pd.concat([pd.Series(analysis_df_1mil['roi'].values, analysis_df_1mil['genre1']),
                       pd.Series(analysis_df_1mil['roi'].values, analysis_df_1mil['genre2']),
                       pd.Series(analysis_df_1mil['roi'].values, analysis_df_1mil['genre3'])])

analysis_df_10mil = analysis_df[analysis_df['working_budget'] >= 10000000].copy()

genre_roi10 = pd.concat([pd.Series(analysis_df_10mil['roi'].values, analysis_df_10mil['genre1']),
                       pd.Series(analysis_df_10mil['roi'].values, analysis_df_10mil['genre2']),
                       pd.Series(analysis_df_10mil['roi'].values, analysis_df_10mil['genre3'])])

## Let's check how many of each genre we have so we can prevent skewing

In [ ]:
pd.DataFrame(genre_roi).reset_index()['index'].value_counts()

In [ ]:
pd.DataFrame(genre_roi1).reset_index()['index'].value_counts()

In [ ]:
pd.DataFrame(genre_roi10).reset_index()['index'].value_counts()

In [ ]:
genre_roi = genre_roi.drop(labels=['Reality-TV', 'Musical'])
genre_roi1 = genre_roi1.drop(labels=['Reality-TV', 'Musical'])
genre_roi10 = genre_roi10.drop(labels=['Western', 'Musical'])



Before we graph these sets, we're going to reset the index and properly name the roi column

In [ ]:
genre_roi_df = pd.DataFrame(genre_roi).reset_index().rename(columns={0:'roi'})
genre_roi1_df = pd.DataFrame(genre_roi1).reset_index().rename(columns={0:'roi'})
genre_roi10_df = pd.DataFrame(genre_roi10).reset_index().rename(columns={0:'roi'})

## General genre return on investment breakdown (no budget floor, excluded genres with fewer than 10 films in dataset)


In [ ]:
my_order = genre_roi_df.groupby("index")["roi"].mean().sort_values().iloc[::-1].index


plt.figure(figsize=(15, 7))

ax = sns.boxplot(x=genre_roi_df['index'], y=genre_roi_df['roi'], data=(genre_roi_df), order=my_order)
plt.setp(ax.get_xticklabels(), rotation=45, fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize = 10)

ax.set_title('Profit by Genre since 2000', fontsize = 17)
ax.set_xlabel('Genre', fontsize=15);
ax.set_ylabel('Return on Investment (in Billions)', fontsize=15);

## General genre return on investment breakdown (budget >$1 million,  excluded genres with fewer than 10 films in dataset)


In [ ]:
my_order = genre_roi1_df.groupby("index")["roi"].mean().sort_values().iloc[::-1].index


plt.figure(figsize=(15, 7))

ax = sns.boxplot(x=genre_roi1_df['index'], y=genre_roi1_df['roi'], data=(genre_roi1_df), order=my_order)
plt.setp(ax.get_xticklabels(), rotation=45, fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize = 10)

ax.set_title('Profit by Genre since 2000 (Budget > $1 Million)', fontsize = 17)
ax.set_xlabel('Genre', fontsize=15);
ax.set_ylabel('Return on Investment (in Billions)', fontsize=15);

## General genre return on investment breakdown (budget >$10 million,  excluded genres with fewer than 10 films in dataset)


In [ ]:
my_order = genre_roi10_df.groupby("index")["roi"].mean().sort_values().iloc[::-1].index


plt.figure(figsize=(15, 7))

ax = sns.boxplot(x=genre_roi10_df['index'], y=genre_roi10_df['roi'], data=(genre_roi10_df), order=my_order)
plt.setp(ax.get_xticklabels(), rotation=45, fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize = 10)

ax.set_title('Profit by Genre since 2000 (Budget > $10 Million)', fontsize = 17)
ax.set_xlabel('Genre', fontsize=15);
ax.set_ylabel('Return on Investment (in millons)', fontsize=15);

In [ ]:
genre_roi2_scifi = pd.DataFrame(pd.concat([pd.Series(analysis_df_1mil.loc[analysis_df_1mil['genre1'] == 'Sci-Fi']['year'].values, analysis_df_1mil.loc[analysis_df_1mil['genre1'] == 'Sci-Fi']['roi']),
                               pd.Series(analysis_df_1mil.loc[analysis_df_1mil['genre2'] == 'Sci-Fi']['year'].values, analysis_df_1mil.loc[analysis_df_1mil['genre2'] == 'Sci-Fi']['roi']),
                               pd.Series(analysis_df_1mil.loc[analysis_df_1mil['genre3'] == 'Sci-Fi']['year'].values, analysis_df_1mil.loc[analysis_df_1mil['genre3'] == 'Sci-Fi']['roi'])]
                              )).reset_index().rename(columns={0:'year'})

grouped_mean = genre_roi2_scifi.groupby('year').mean()

ax = sns.lineplot(x= grouped_mean.index, y=grouped_mean.roi, data=grouped_mean)
plt.ylim(0, 450000000)
ax.set_title('Mean Return on Investment', fontsize = 17)
ax.set_xlabel('Genre', fontsize=15);
ax.set_ylabel('Return on Investment (in millons)', fontsize=15);

In [ ]:
working_df[working_df['genres'].str.contains('Sci-Fi')]

## group by runtime_minutes


In [ ]:
analysis_df_runtime_grouped = analysis_df_1mil.copy()

In [ ]:
analysis_df_runtime_grouped['runtime_desc'] = analysis_df_runtime_grouped.apply(
    lambda x: 'short' if x['runtime_minutes'] <= 95.0 else(
        'medium' if x['runtime_minutes'] <= 118.0 else 'long'), axis = 1
)

In [ ]:
ax = sns.boxplot(x=analysis_df_runtime_grouped['runtime_desc'], y=analysis_df_runtime_grouped['roi'], data=analysis_df_runtime_grouped)


## Looking for trends in Adventure/Sci-Fi

In [ ]:
analysis_df_1mil.loc[((analysis_df_1mil['genre3'] == 'Sci-Fi') |
                  (analysis_df_1mil['genre2'] == 'Sci-Fi') |
                  (analysis_df_1mil['genre1'] == 'Sci-Fi') )&(
                    (analysis_df_1mil['genre3'] == 'Adventure') |
                    (analysis_df_1mil['genre2'] == 'Adventure') |
                    (analysis_df_1mil['genre1'] == 'Adventure'))
                 ].sort_values(by = 'roi', ascending=False).reset_index().style.apply(
                lambda x: ['background: #00a4ef' if x.title == 'Inception' or x.title == 'Interstellar' else '' for i in x], axis = 1)

Christopher Nolan movies are the only non-lisenced films on the list until number 49 - Paul